<a href="https://colab.research.google.com/github/KleytonFarias/ProjetoImersaoAluraIA/blob/main/Projeto_Imersao_IA_ALURA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AQUI ESTÁ O CÓDIGO DE INSTALAÇÃO DO GEMINI

In [1]:
!pip install google.generativeai

AQUI É ONDE EU FAÇO A IMPORTAÇÃO DAS BIBLIOTECAS E A CHAMADA DA API USANDO A CHAVE

In [52]:
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display
from google.colab import userdata
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Configurar API Key
API_KEY = userdata.get('SECRETKEY')
genai.configure(api_key=API_KEY)

AQUI É ONDE EU ADICIONO AS INFORMAÇÕES EM RELAÇÃO AO CONTRATO E AS REGRAS

In [53]:
# AQUI É FEITA A BUSCA DO MODELO
for m in genai.list_models():
    if 'embedContent' in m.supported_generation_methods:
        EMBEDDING_MODEL_NAME = m.name
        break

if EMBEDDING_MODEL_NAME is None:
    raise ValueError("Nenhum modelo com suporte a 'embedContent' encontrado!")

# AQUI FICA AS INSERÇÕES DOS CONTEUDOS DAS VARIAVEIS
regras_input = widgets.Textarea(description='Regras (separadas por linha):', value='')
contrato_input = widgets.Textarea(description='Contrato:', value='')
botao_submit = widgets.Button(description='Analisar Contrato')

AQUI É A PARTE QUE CRIA AS FUNÇÕES E FAZ A ANÁLISE DO CONTRATO DE ACORDO COM AS REGRAS ESTABELECIDAS.

In [54]:
relatorio = []

def gerar_embedding(texto):
    """Gera um embedding para o texto usando 'embedContent'."""
    response = genai.embed_content(model=EMBEDDING_MODEL_NAME, content=texto)
    return response['embedding']

def calcular_similaridade(embedding1, embedding2):
    """Calcula a similaridade de cosseno entre dois embeddings."""
    embedding1 = np.array(embedding1)
    embedding2 = np.array(embedding2)
    return cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]

def analisar_secao(secao, regras):
    """Analisa uma seção do contrato em relação às regras."""
    secao_embedding = gerar_embedding(secao)
    resultados = []

    for i, regra in enumerate(regras):
        regra_embedding = gerar_embedding(regra)
        similaridade = calcular_similaridade(secao_embedding, regra_embedding)

        resultados.append({
            'regra': regra,
            'similaridade': similaridade
        })

    return resultados

def on_submit(button):
    global relatorio
    regras = regras_input.value.splitlines()
    contrato = contrato_input.value.splitlines()

    for i, secao in enumerate(contrato):
        resultados = analisar_secao(secao, regras)
        relatorio.append(f"**Seção {i+1}:**")

        for resultado in resultados:
            relatorio.append(f"  - Regra: {resultado['regra']}")
            relatorio.append(f"    Similaridade: {resultado['similaridade']:.2f}")

            if resultado['similaridade'] < 0.6:
                relatorio.append(f"    **Atenção:** Baixa similaridade com a regra!")

    print("\n".join(relatorio))

botao_submit.on_click(on_submit)
display(regras_input)
display(contrato_input)
display(botao_submit)

Textarea(value='', description='Regras (separadas por linha):')

Textarea(value='', description='Contrato:')

Button(description='Analisar Contrato', style=ButtonStyle())

**Seção 1:**
  - Regra: O contrato deve ter duração mínima de 1 ano.
    Similaridade: 0.93
  - Regra: O valor da remuneração deve ser explicitamente definido.
    Similaridade: 0.71
  - Regra: O contrato deve especificar as obrigações de ambas as partes.
    Similaridade: 0.80
**Seção 2:**
  - Regra: O contrato deve ter duração mínima de 1 ano.
    Similaridade: 0.74
  - Regra: O valor da remuneração deve ser explicitamente definido.
    Similaridade: 0.74
  - Regra: O contrato deve especificar as obrigações de ambas as partes.
    Similaridade: 0.73
**Seção 3:**
  - Regra: O contrato deve ter duração mínima de 1 ano.
    Similaridade: 0.76
  - Regra: O valor da remuneração deve ser explicitamente definido.
    Similaridade: 0.74
  - Regra: O contrato deve especificar as obrigações de ambas as partes.
    Similaridade: 0.88
